# The Sequential Model API

> In this post, we will dig into the basic usage of building neural network with Tensorflow Sequential APIs.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, Deep_Learning, Tensorflow-Keras]
- image: 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
print("NumPy: {}".format(np.__version__))
print("Pandas: {}".format(pd.__version__))
print("Tensorflow: {}".format(tf.__version__))

NumPy: 1.18.1
Pandas: 1.0.1
Tensorflow: 2.3.1


## Building a sequential model


### Coding Tutorial - Build a feedforward neural network model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Softmax

In [4]:
# Build the Sequential feedforward neural network model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Softmax()
])

In [5]:
# Print the model init weights
model.weights

[<tf.Variable 'dense/kernel:0' shape=(784, 16) dtype=float32, numpy=
 array([[ 0.07387501, -0.05956989,  0.07959732, ...,  0.05007657,
         -0.05077644,  0.03850707],
        [ 0.04580013,  0.03747427, -0.07693   , ...,  0.07467815,
          0.03879775, -0.06678017],
        [ 0.07918745,  0.06736486, -0.01312144, ..., -0.04416531,
          0.03269102,  0.03084324],
        ...,
        [-0.04102856, -0.00359754,  0.0198087 , ...,  0.08463748,
         -0.01913959,  0.03067476],
        [ 0.07913649, -0.08255742, -0.01171618, ...,  0.041526  ,
          0.07023814, -0.08628033],
        [ 0.02095706,  0.06044121, -0.06647719, ..., -0.0039745 ,
         -0.0590837 ,  0.07673346]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(16,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(16, 16) dtype=float32, numpy=
 array([[-0.35056257,  0.11791721,  0.23483494, -0.4277878

In [6]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                12560     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
softmax (Softmax)            (None, 16)                0         
Total params: 13,104
Trainable params: 13,104
Non-trainable params: 0
_________________________________________________________________


## Convolutional and pooling layers
### Coding Tutorial - Build Convolutional Neural network model


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [8]:
# Build the Sequential convolutional neural network model
model = Sequential([
    Conv2D(16, (3, 3), padding='SAME', strides=2, activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((3, 3)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 17,258
Trainable params: 17,258
Non-trainable params: 0
_________________________________________________________________


Currently, the default tensorflow data type is 'channels_last', but we can change it with `data_format` argument.

In [10]:
model = Sequential([
    Conv2D(16, (3, 3), padding='SAME', strides=2, activation='relu', input_shape=(1, 28, 28), data_format='channels_first'),
    MaxPooling2D((3, 3), data_format='channels_first'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 14, 14)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 4, 4)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                650       
Total params: 17,258
Trainable params: 17,258
Non-trainable params: 0
_________________________________________________________________


## Weight and bias initialisers 

In this reading, we investigate different ways to initialise weights and biases in the layers of neural networks.

### Default weights and biases

In the models we have worked with so far, we have not specified the initial values of the weights and biases in each layer of our neural networks.

The default values of the weights and biases in TensorFlow depend on the type of layers we are using. 

For example, in a `Dense` layer, the biases are set to zero (`zeros`) by default, while the weights are set according to `glorot_uniform`, the Glorot uniform initialiser. 

The Glorot uniform initialiser draws the weights uniformly at random from the closed interval $[-c,c]$, where $$c = \sqrt{\frac{6}{n_{input}+n_{output}}}$$

and $n_{input}$ and $n_{output}$ are the number of inputs to, and outputs from the layer respectively.

### Initialising your own weights and biases
We often would like to initialise our own weights and biases, and TensorFlow makes this process quite straightforward.

When we construct a model in TensorFlow, each layer has optional arguments `kernel_initialiser` and `bias_initialiser`, which are used to set the weights and biases respectively.

If a layer has no weights or biases (e.g. it is a max pooling layer), then trying to set either `kernel_initialiser` or `bias_initialiser` will throw an error.

Let's see an example, which uses some of the different initialisations available in Keras.

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPooling1D

# Construct a model
model = Sequential([
    Conv1D(filters=16, kernel_size=3, input_shape=(128, 64), kernel_initializer='random_uniform',
           bias_initializer='zeros', activation='relu'),
    MaxPooling1D(pool_size=4),
    Flatten(),
    Dense(64, kernel_initializer='he_uniform', bias_initializer='ones', activation='relu')
])

As the following example illustrates, we can also instantiate initialisers in a slightly different manner, allowing us to set optional arguments of the initialisation method.